In [1]:
# load dependencies
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

# data libraries
import yfinance as yf
import quantstats as qs

In [25]:
# pull data for S&P 500 and Treasury from Yahoo Finance
ticker_list = ['SPY', 'TLT']
financial_data = yf.download(ticker_list, start = '2020-01-01', end ='2025-01-01', auto_adjust=False, group_by='ticker')

# change the multiIndex format to single column data frame for ease of usage
fin_tidy_data = financial_data.stack(level=0).reset_index()
fin_tidy_data = fin_tidy_data.rename(columns={'level_1': 'Ticker'})
fin_tidy_data['Date'] = pd.to_datetime(fin_tidy_data['Date'])
fin_tidy_data = fin_tidy_data.set_index('Date')
fin_tidy_data = fin_tidy_data.sort_values(by=['Ticker', 'Date'])

fin_tidy_data

[*********************100%***********************]  2 of 2 completed


Price,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-01-02,SPY,323.540009,324.890015,322.529999,324.869995,300.291626,59151200
2020-01-03,SPY,321.160004,323.640015,321.100006,322.410004,298.017700,77709700
2020-01-06,SPY,320.489990,323.730011,320.359985,323.640015,299.154602,55653900
2020-01-07,SPY,323.019989,323.540009,322.239990,322.730011,298.313507,40496400
2020-01-08,SPY,322.940002,325.779999,322.670013,324.450012,299.903320,68296000
...,...,...,...,...,...,...,...
2024-12-24,TLT,87.040001,87.889999,86.980003,87.870003,86.651482,22377600
2024-12-26,TLT,87.209999,87.959999,87.199997,87.820000,86.602165,19981800
2024-12-27,TLT,87.480003,87.779999,87.059998,87.099998,85.892151,27262300


In [26]:
# Calculate simple returns and log returns per ticker
fin_tidy_data['Return'] = fin_tidy_data.groupby('Ticker')['Adj Close'].pct_change()
fin_tidy_data['Log_Return'] = np.log1p(fin_tidy_data['Return'])

# Calculate Value at Risk (VaR) per ticker
var_values = fin_tidy_data.groupby('Ticker')['Return'].apply(qs.stats.var)

# Merge VaR into the main DataFrame (optional, if you want it repeated per row)
fin_tidy_data = fin_tidy_data.merge(var_values.rename('Value_at_Risk'), on='Ticker')

fin_tidy_data

,Ticker,Open,High,Low,Close,Adj Close,Volume,Return,Log_Return,Value_at_Risk
0,SPY,323.540009,324.890015,322.529999,324.869995,300.291626,59151200,NaN,NaN,-0.021131
1,SPY,321.160004,323.640015,321.100006,322.410004,298.017700,77709700,-0.007572,-0.007601,-0.021131
2,SPY,320.489990,323.730011,320.359985,323.640015,299.154602,55653900,0.003815,0.003808,-0.021131
3,SPY,323.019989,323.540009,322.239990,322.730011,298.313507,40496400,-0.002812,-0.002816,-0.021131
4,SPY,322.940002,325.779999,322.670013,324.450012,299.903320,68296000,0.005329,0.005315,-0.021131
...,...,...,...,...,...,...,...,...,...,...
2511,TLT,87.040001,87.889999,86.980003,87.870003,86.651482,22377600,0.004229,0.004220,-0.018799
2512,TLT,87.209999,87.959999,87.199997,87.820000,86.602165,19981800,-0.000569,-0.000569,-0.018799
2513,TLT,87.480003,87.779999,87.059998,87.099998,85.892151,27262300,-0.008199,-0.008232,-0.018799
2514,TLT,87.830002,88.040001,87.669998,87.800003,86.582451,48519600,0.008037,0.008005,-0.018799
